In [58]:
from nltk.corpus import stopwords
import gensim
import pickle
import spacy
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import pandas as pd
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [118]:
data_words = list(sent_to_words(df_list))

In [119]:
 data_words_nostops = remove_stopwords(data_words)
    # Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_trigrams = make_trigrams(data_words_bigrams)
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
    vocab=list(id2word.values()) #all vocabulary used in lda model, we need to exclude new words from unseen article so that lda works without errors
    def remove_new(lemma_list):
        return [word for word in lemma_list if word in vocab]
    data_lemmatized= [remove_new(x)for x in data_lemmatized]

In [ ]:
%%time
corpus = [id2word.doc2bow(text) for text in data_lemmatized]

[['bake',
  'adikri',
  'go',
  'air_strikes',
  'terrorist_camp',
  'balakot',
  'kashmir',
  'response',
  'slain_soldier',
  'pulwama',
  'strike',
  'jammu',
  'kashmir',
  'responsibility',
  'attack',
  'pakistani_militant',
  'group',
  'take',
  'jaish_mohammed',
  'attack',
  'india',
  'become',
  'war',
  'position',
  'take',
  'modi',
  'cabinet',
  'government',
  'eliminate',
  'temporary',
  'restrictions_imposed',
  'indian_airspace',
  'february',
  'highly_placed_source',
  'say',
  'begin',
  'gesture',
  'india',
  'open',
  'flight',
  'airspace',
  'ago',
  'pak',
  'haul',
  'flight',
  'airspace',
  'open',
  'delhi',
  'west',
  'include',
  'south_asia',
  'even',
  'lower',
  'infarmeshn',
  'temporary',
  'restriction',
  'air',
  'route',
  'indian_airspace',
  'impose',
  'indian_air_force',
  'feb',
  'remove',
  'mds',
  'indian_air_force',
  'may',
  'source',
  'say',
  'usually',
  'indication',
  'country',
  'willing',
  'lift',
  'ban',
  'take',


In [114]:
#df list is the list of new atricles.
#EX:df_list=["MUMBAI: Bollywood actress Parineeti Chopra these days is enough remains in the spotlight", "Thereby Chopra these days are busy biopic of shuttler Saina Nehwal. For fits the role of Saina are thereby Chopra hard. Tell the same to thereby Chopra and Sania Mirza Best Friend. Both participated in the recent chat Neha Dhupia show. Thereby Chopra at the revelations involving his professional and personal life. During the rapid fire round Neha asked a funny question thereby. The question that Siddharth Malhotra holds and who better kisser than Arjun Kapoor. Thereby Chopra in response said, Arjun is a better kisser. The unique and rare relationship with my Arjun. I would like here with Arjun. Arjun and then choosing one of Siddhartha I'd like to name Arjun. & # 39; & # 39; Said thereby show that Deepika Padukone and Amitabh Bachchan blockbuster 'Piku & # 39; Before Deepika they had offered. But to date the film because of the issue. After Deepika got the roll"]
def update_lda(df_list, model_path):#path:"./lda_temp/lda15top"
    trigram_model=gensim.models.phrases.Phraser.load("./lda_model_data/trigram_model")
    bigram_model=gensim.models.phrases.Phraser.load("./lda_model_data/bigram_model")
    stop_words = stopwords.words('english')
    stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
            #python3 -m spacy download en
            #1 time download
    nlp = spacy.load('en', disable=['parser', 'ner'])
            # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
    def make_bigrams(texts):
        return [bigram_model[doc] for doc in texts]

    def make_trigrams(texts):
        return [trigram_model[bigram_model[doc]] for doc in texts]
    def remove_stopwords(texts):
                return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent))
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out
    data_words = list(sent_to_words(df_list))
    data_words_nostops = remove_stopwords(data_words)
    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)
    data_words_trigrams = make_trigrams(data_words_bigrams)
    data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    del data_words_bigrams
    del data_words_trigrams
    #del trigram_mod
    del data_words_nostops
    #del stop_words
    #now id2word dictionary cannot be changed so we must remove new words before giving it to lda model.
    file = open("./lda_model_data/id2word_dict.pickle", 'rb')
            #load  information from that file
    id2word = pickle.load(file)
            # close the file
    file.close()
    vocab=set(id2word.values()) #all vocabulary used in lda model, we need to exclude new words from unseen article so that lda works without errors
    def remove_new(lemma_list):
        return [word for word in lemma_list if word in vocab]
    data_lemmatized= [remove_new(x)for x in data_lemmatized]
    corpus = [id2word.doc2bow(text) for text in data_lemmatized]
    #list of tuples for each doc- (id,frequency) of word in that document
    optimal_model=gensim.models.ldamodel.LdaModel.load(model_path)#"./lda_temp/lda15top"
    optimal_model.update(corpus)
    optimal_model.save(model_path)

In [83]:
database=pd.read_pickle("database_for_doc2vec.pickle")

In [70]:
len(database)

87431

In [71]:
database=database.append(june_data, ignore_index=True)

/home/psdhruv/python-environments/env3/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [72]:
len(database)

100926

In [76]:
#DOC2VEC update
#bring the new data in above format and append that dataframe with the old dataframe to retrain the doc2vec model.
#2 files will be updated. data file and doc2vec model file.
#new data will have columns news_id, crtd_date, upv3day, english so as database.
def doc2vec_update(new_data):
    database=pd.read_pickle("database_for_doc2vec.pickle")#it has all article content with views
    database=database.append(new_data, ignore_index=True)
    database=database.sort_values(by="news_id", ascending=True).reset_index(drop=True)
    df_list=list(database["english"])
    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
    data_words = list(sent_to_words(df_list))
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(data_words)]
    model = Doc2Vec(vector_size=25, min_count=0,workers=2, alpha=0.025, min_alpha=0.025, dm=0, epochs=1)
    model.build_vocab(documents)
    for epoch in range(10):
        if epoch%2==0:
            print("training {} ".format(epoch))
        model.train(documents, total_examples=model.corpus_count,epochs=model.epochs)
        model.alpha-=0.002
        model.min_alpha= model.alpha#fixing learning rate throughout 1 epoch
    model.save("./lda_model_data/doc2vec_model")
    #update the 'data' dataframe to be sent over cloud function.(it will have latest indexes with updated data.)
    data=database[["crtd_date", "news_id", "upv3day"]]
    data.to_pickle("./lda_model_data/data.pickle")
    database.to_pickle("database_for_doc2vec.pickle)



In [59]:
#dnnregression model update
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pandas as pd
import tensorflow as tf
import shutil
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)


In [56]:
CSV_COLUMNS=['upv3day', 'polarity', 'subjectivity', 'num_of_sentences',
       'length', 'num_of_identities', 'other_entities', 'upv3hr', 'msaav10', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14',
       'cat', 'weekday', 'hour']
FEATURES = CSV_COLUMNS[1:]
LABEL = CSV_COLUMNS[0]
#deFAULTS DEFINES datatype of column in tf.decode_csv
DEFAULTS = [[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],["none"],[0],[0]]

def update_dnn(num_epochs,new_data_path, batchsize,modelpath)#ex:'./final_data2/df_valid.csv', file must be in csv.It should have all columns defined in columns.
#set batchsize equals to number of rows in new data if data is not too large.(<50000)
    def read_dataset(filename, epochs,batch_size = 10000):
       def _input_fn():
          def decode_csv(row):
            columns = tf.decode_csv(row, record_defaults = DEFAULTS)
            features = dict(zip(CSV_COLUMNS, columns))
            label = features.pop(LABEL) # remove label from features and store
            return features, label

          # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
          filenames_dataset = tf.data.Dataset.list_files(filename, shuffle=False)
          # Read lines from text files
          textlines_dataset = filenames_dataset.flat_map(tf.data.TextLineDataset).skip(count=1)
          # Parse text lines as comma-separated values (CSV)
          dataset = textlines_dataset.map(decode_csv)
          dataset = dataset.shuffle(buffer_size = batch_size, seed=2)#or 100 * batch_size(if smaller batch size than training)
          # Note:
          # use tf.data.Dataset.flat_map to apply one to many transformations (here: filename -> text lines)
          # use tf.data.Dataset.map      to apply one to one  transformations (here: text line -> feature list)

    #       if mode == tf.estimator.ModeKeys.TRAIN:
    #           num_epochs = None # loop indefinitely, so limit will be defined by max steps in estimator
    #           dataset = dataset.shuffle(buffer_size = 70000, seed=2)#100 * batch_size
    #       else:
    #           num_epochs = None#1 # end-of-input after this

          dataset = dataset.repeat(epochs).batch(batch_size)

          return  dataset.make_one_shot_iterator().get_next()
       return _input_fn




    def get_train_input_fn():
      return read_dataset(new_data_path, epochs=num_epochs,batch_size=batchsize)#use batch_size=length of new dataset


    def make_feature_cols():#no y here , must enclose cat columns in indentity or embedding cols for dnn
        categorical_column1=tf.feature_column.categorical_column_with_vocabulary_list(key='cat', vocabulary_list=('blogs','business','dharm','education-and-jobs','entertainment','international','life-style','nari','national','other','regional','sports'))
        categorical_column2=tf.feature_column.categorical_column_with_identity(key='weekday', num_buckets=7)
        categorical_column3=tf.feature_column.bucketized_column(source_column=tf.feature_column.numeric_column("hour"), boundaries=[6,12,16,20])
        input_columns = [tf.feature_column.numeric_column('polarity'),
                         tf.feature_column.numeric_column('subjectivity'),
                         tf.feature_column.numeric_column('num_of_sentences'),
                         tf.feature_column.numeric_column('length'),
                         tf.feature_column.numeric_column('num_of_identities'),
                         tf.feature_column.numeric_column('other_entities'),
                         tf.feature_column.numeric_column('upv3hr'),
                         tf.feature_column.numeric_column('msaav10'),
                         tf.feature_column.numeric_column('0'),
                         tf.feature_column.numeric_column('1'),
                         tf.feature_column.numeric_column('2'),
                         tf.feature_column.numeric_column('3'),
                         tf.feature_column.numeric_column('4'),
                         tf.feature_column.numeric_column('5'),
                         tf.feature_column.numeric_column('6'),
                         tf.feature_column.numeric_column('7'),
                         tf.feature_column.numeric_column('8'),
                         tf.feature_column.numeric_column('9'),
                         tf.feature_column.numeric_column('10'),
                         tf.feature_column.numeric_column('11'),
                         tf.feature_column.numeric_column('12'),
                         tf.feature_column.numeric_column('13'),
                         tf.feature_column.numeric_column('14'),
                         tf.feature_column.embedding_column(categorical_column1,dimension=10),

                         tf.feature_column.indicator_column(categorical_column2),
                         tf.feature_column.indicator_column(categorical_column3)]
        return input_columns

    def serving_input_fn():#actual datatype in data for serving
        feature_placeholders =  {
            column.name: tf.placeholder(tf.float32, [None]) for column in list(make_feature_cols())[:23]
        }

        feature_placeholders["cat"]=tf.placeholder(tf.string, [None])
        feature_placeholders["weekday"]=tf.placeholder(tf.int32, [None])
        feature_placeholders["hour"]=tf.placeholder(tf.int32, [None])


        features = feature_placeholders
        return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


    def my_metric(labels, predictions):
        pred_values = predictions['predictions']
        return {'metric1': tf.metrics.root_mean_squared_error(labels, pred_values)}

    run_config = tf.estimator.RunConfig( save_checkpoints_secs = 5, keep_checkpoint_max = 4, tf_random_seed=100)


    def train_and_evaluate(num_train_steps):
        #tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
        estimator = tf.estimator.DNNRegressor(
            config = run_config,
            model_dir = modelpath #"./cloudmlemodels",
            feature_columns = make_feature_cols(),
            hidden_units = [100,90,81,73],
            optimizer=tf.train.AdagradOptimizer(learning_rate=0.005))
        estimator = tf.contrib.estimator.add_metrics(estimator, my_metric)
        train_spec = tf.estimator.TrainSpec(
            input_fn =get_train_input_fn(),
             max_steps = num_train_steps)
        exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
        eval_spec = tf.estimator.EvalSpec(
            input_fn = get_train_input_fn(),
            steps = None,#none will evaluate whole eval data in each test
            start_delay_secs = 10,
            throttle_secs = 10,
            exporters = exporter)
        tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    train_and_evaluate(None)#None max steps will put no imit on max steps so till the input funbction is providing data it will continue training.
    #so num of epochs will be decided merely by input dataset epochs.

In [60]:
#Example of updating models:
june=pd.read_pickle("data_final_june.pickle")

In [64]:
june_articles=pd.read_pickle("june_articles.pickle")

In [78]:
june_data

,news_id,crtd_date,upv3h,upv3d,english
0,1003745,2019-06-01 00:32:22,27.0,89.0,NEW DELHI: The Indian Air Force by baking Adik...
1,1003746,2019-06-01 00:52:05,18.0,55.0,Bangalore: Lok Sabha after the defeat in the e...
2,1003747,2019-06-01 00:53:23,1.0,17.0,"Today the water of the country's many rivers, ..."
3,1003748,2019-06-01 01:10:30,3.0,41.0,Prime Minister Narendra Modi its neighbors hav...
4,1003749,2019-06-01 01:21:44,3.0,32.0,MUMBAI: Bollywood filmmaker Boney Kapoor remai...
5,1003750,2019-06-01 01:51:26,19.0,61.0,New Delhi / Jalandhar (Web Desk): The first me...
6,1003751,2019-06-01 02:08:04,2.0,13.0,Prime Minister of India by inviting 30 May the...
7,1003752,2019-06-01 02:11:31,12.0,36.0,MUMBAI: Bollywood actor Salman Khan's movie on...
8,1003753,2019-06-01 03:05:25,2.0,59.0,After nearly 52 years in Sena (became party in...
9,1003754,2019-06-01 04:04:45,25.0,214.0,National Desk: center after the formation of t...


In [65]:
june_data=pd.merge(june, june_articles, on=["news_id"])

In [86]:
june_data.drop(columns=["upv3h"], inplace=True)

In [89]:
june_data=june_data.rename(columns={"upv3d":"upv3day"})

In [109]:
#['news_id', 'crtd_date', 'upv3day', 'english'] we need these columns only while updating doc2vec.

In [77]:
doc2vec_update(june_data)

/home/psdhruv/python-environments/env3/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
W0719 07:07:07.925075 140258506000128 base_any2vec.py:723] consider setting layer size to a multiple of 4 for greater performance


training 0 
training 2 
training 4 
training 6 
training 8 


/home/psdhruv/python-environments/env3/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [110]:
df_list=june_data["english"]

In [112]:
df_list

0        NEW DELHI: The Indian Air Force by baking Adik...
1        Bangalore: Lok Sabha after the defeat in the e...
2        Today the water of the country's many rivers, ...
3        Prime Minister Narendra Modi its neighbors hav...
4        MUMBAI: Bollywood filmmaker Boney Kapoor remai...
5        New Delhi / Jalandhar (Web Desk): The first me...
6        Prime Minister of India by inviting 30 May the...
7        MUMBAI: Bollywood actor Salman Khan's movie on...
8        After nearly 52 years in Sena (became party in...
9        National Desk: center after the formation of t...
10       National Desk: New Foreign Minister also chall...
11       NEW DELHI: The Delhi government is women prepa...
12       What is Saw (Video) Today Saturday date thrayo...
13       They saw what looked (Video) June 1, 2019 Satu...
14       Hoshiarpur / Pathankot (Sharda) are graceful a...
15       Chandigarh (Bhullar): Punjab Speaker Rana KP S...
16       Amritsar / Tahli Sahib (Arun, Baljeet): Sri An.

In [115]:
update_lda(df_list, "./lda_temp/lda15top")

/home/psdhruv/python-environments/env3/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


KeyboardInterrupt: 